# Agent with Memory

The intution behind a simple agent architecture is **ReAct**. It does the following operations: 
- `act`: Performs the relevant tool call 
- `observe`: Post tool executed successfully, it returns back the response to the LLM
- `reason`: LLM goes through the tool output, and reasons whether it needs to perform another tool call or generates its final response. 

**FYI** - For every graph execution when we call the `graph.invoke(dict)` function, the state is transient (temporary) and there will be no persistence. Hence, if we run the graph next time it doesn't have the previous interaction memory stored.

To solve this problem, Langgraph has in-built **Memory Saver**, which saves every graph execution details like ids, state data, etc. in form of checkpoints. All these checkpoints are collectively stored in `threads`. 

Each thread has an `id` assigned, and in-short thread is the collection of checkpoints.

For example, in our case:

1. **First graph execution** - Graph runs "add 2 and 3" then it returns 5 with one tool call.

2. **Second graph execution** - Graph runs "multiply that with 5" now it might hallucinate it doesn't know what is "that" referring to as it doesn't have previous grapg execution memory.

## Loading Environment Variables

In [4]:
from dotenv import load_dotenv
_ = load_dotenv()

## Building Agent
 
We will be re-using the code written for developing Router, and as learnt about the "Agent", once the router routes the control to the tool node, the output of the tool node will be sent back to the tool_calling_llm, where the model observes and reasons whether it needs to do another tool call or not required. 

In [5]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

from typing import TypedDict
from langchain_core.messages import AIMessage, HumanMessage, AnyMessage, SystemMessage

from langchain_groq import ChatGroq

# Define the LLM model 
llm = ChatGroq(model="qwen/qwen3-32b")

# Define the Messages State 
class MessagesState(MessagesState):
    pass # add any other variables you need here

# Defining tools 
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [multiply, add, divide]

# Binding tools to the LLM
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

# Define tool_calling_llm node 
def tool_calling_llm(state: MessagesState):
    systemMessage = SystemMessage(content="You are a helpful assistant that can perform arithmetic operations.")
    return {"messages" : [llm_with_tools.invoke([systemMessage] + state["messages"])]} 

# Define the graph
builder = StateGraph(MessagesState) 

# adding nodes 
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

# adding edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition) # This will call the tools if the LLM asks for it.
builder.add_edge("tools", "tool_calling_llm") # This will return to the LLM after the tool is called as per ReAct pattern. (Simple Agent)

## Adding Memory Saver

In [13]:
from langgraph.checkpoint.memory import MemorySaver 
memory = MemorySaver() 

# Building and compiling graph with MemorySaver
react_graph = builder.compile(checkpointer=memory)

# Creating a thread which stores all the checkpoints inside
config = {"configurable" : {"thread_id": "cde"}}

## Invoking the graph with config

In [14]:
messages = [HumanMessage(content="Add 3 and 4")]
response = react_graph.invoke({"messages": messages}, config=config) # Adding the config

for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4
================================== Ai Message ==================================
Tool Calls:
  add (j8t7qm1ww)
 Call ID: j8t7qm1ww
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is $\boxed{7}$.


In [15]:
messages = [HumanMessage(content="Multiply that with 10")]
response = react_graph.invoke({"messages": messages}, config=config) # Adding the config

for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4
================================== Ai Message ==================================
Tool Calls:
  add (j8t7qm1ww)
 Call ID: j8t7qm1ww
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is $\boxed{7}$.
================================ Human Message =================================

Multiply that with 10
================================== Ai Message ==================================
Tool Calls:
  multiply (18wrrh5wa)
 Call ID: 18wrrh5wa
  Args:
    a: 7
    b: 10
================================= Tool Message =================================
Name: multiply

70
================================== Ai Message ==================================

The product of 7 and 10 is $\boxed{70}$.
